In [1]:
import pandas as pd

PATH = "."
elections_data_path = f"{PATH}/data/elections.csv"

# Séparer les caractéristiques et les étiquettes
elections_data = pd.read_csv(elections_data_path)
X = elections_data.drop("Label", axis=1)
y = elections_data["Label"]

elections_data

,Adresse,Majeur?,Nationalite,Label
0,Paris,oui,Francais,1
1,Paris,non,Francais,-1
2,Montpellier,oui,Italien,1
3,Paris,oui,Suisse,-1
4,Strasbourg,non,Italien,-1
5,Strasbourg,non,Francais,-1
6,Strasbourg,oui,Francais,1
7,Montpellier,oui,Suisse,-1


In [2]:
from collections import Counter
from copy import copy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


class SymbolicDecisionTree:
    def __init__(self):
        self.tree = None
        self.data = None

    class Node:
        def __init__(
            self,
            feature=None,
            value=None,
            true_branch=None,
            false_branch=None,
            result=None,
            entropy=None,
        ):
            self.feature = feature
            self.value = value
            self.true_branch = true_branch
            self.false_branch = false_branch
            self.result = result
            self.entropy = entropy

    def fit(self, X, y):
        # Combine features and labels
        training_data = pd.concat([X, y], axis=1)
        self.data = training_data
        self.tree = self.build_tree(training_data)

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            predictions = [
                self.predict_single_entry(entry) for _, entry in X.iterrows()
            ]
        elif isinstance(X, pd.Series):
            predictions = self.predict_single_entry(X)
        else:
            raise TypeError(
                f"type {type(X)} not supported in predict, please implement or convert to a pandas object"
            )
        return predictions

    def predict_xai(self, entry):
        explanation, _ = self.trace_tree(entry, self.tree, explanation=[])
        return explanation

    def counter_fact(self, entry):
        explanation, _ = self.untrace_tree(entry, self.tree, explanation=[])
        return explanation

    def build_tree(self, data):
        # If there's no data, or if all targets are the same,
        # return a leaf node with the result
        if len(data) == 0:
            return self.Node()

        current_uncertainty = self.gini(data.iloc[:, -1])
        # print(data.iloc[:, -1])
        best_gain = 0
        best_criteria = None
        best_sets = None

        feature_count = len(data.columns) - 1  # number of attributes

        for col in range(feature_count):  # for each feature
            feature_values = data.iloc[:, col].unique()  # unique values
            for val in feature_values:  # for each value
                partitioned_data = self.partition(data, data.columns[col], val)

                # Information gain
                p = float(partitioned_data[0].shape[0]) / data.shape[0]
                gain = (
                    current_uncertainty
                    - p * self.gini(partitioned_data[0].iloc[:, -1])
                    - (1 - p) * self.gini(partitioned_data[1].iloc[:, -1])
                )

                if (
                    gain > best_gain
                    and len(partitioned_data[0]) > 0
                    and len(partitioned_data[1]) > 0
                ):
                    best_gain = gain
                    best_criteria = (data.columns[col], val)
                    best_sets = partitioned_data

        if best_gain > 0:
            true_branch = self.build_tree(best_sets[0])
            false_branch = self.build_tree(best_sets[1])
            return self.Node(
                feature=best_criteria[0],
                value=best_criteria[1],
                true_branch=true_branch,
                false_branch=false_branch,
                entropy=current_uncertainty,
            )
        else:
            # We're at a leaf, determine the outcome most frequent class
            outcome = data.iloc[:, -1].value_counts().idxmax()
            return self.Node(result=outcome, entropy=current_uncertainty)

    def predict_single_entry(self, entry):
        node = self.tree
        while node.result is None:
            if entry[node.feature] == node.value:
                node = node.true_branch
            else:
                node = node.false_branch
        return node.result

    def trace_tree(self, entry, node, explanation):
        if node.result is not None:
            return explanation, node.result

        if entry[node.feature] == node.value:
            explanation.append(f"{node.feature} == {node.value}")
            return self.trace_tree(entry, node.true_branch, explanation)
        else:
            explanation.append(f"{node.feature} != {node.value}")
            return self.trace_tree(entry, node.false_branch, explanation)

    @staticmethod
    def partition(data, feature, value):
        true_data = data[data[feature] == value]
        false_data = data[data[feature] != value]
        return true_data, false_data

    @staticmethod
    def gini(labels):
        impurity = 1
        label_counts = Counter(labels)
        for label in label_counts:
            prob_of_label = label_counts[label] / float(len(labels))
            impurity -= prob_of_label**2
        return impurity

    def display_tree(self, node=None, indent="", branch=""):
        """
        Visualise la structure arborescente de l'arbre dans un format clair et organisé.

        Format de sortie :
        "├──" indique un nœud qui confirme la condition.
        "└──" indique un nœud qui ne confirme pas la condition.
        Les feuilles affichent le résultat de la classe.
        Les nœuds internes affichent le critère de décision et son entropie.

        Exemple :
        feature_name1 == feature_value1? (Entropy = 0.1234)
        ├── Class : Class_A
        └── feature_name2 == feature_value2? (Entropy = 0.5678)
            ├── Class : Class_B
            └── Class : Class_C

        Dans cet exemple :
        - Si feature_name1 == feature_value1, l'arbre de décision le classe comme
        'Class_A'.
        - Sinon, il vérifie si feature_name2 == feature_value2 ; si c'est vrai, il est
        classé comme 'Class_B', et si c'est faux, il est classé comme 'Class_C'.
        """

        if node is None:
            node = self.tree

        # Base case: if it's a leaf node, print the result and return
        if node.result is not None:
            print(f"{indent}{branch}Class: {node.result}")
            return

        # Print the criterion for the current node
        print(
            f"{indent}{branch}{node.feature} == {node.value}? (Entropy = {node.entropy:.4f})"
        )

        # Recursive case: print the true and false branches
        new_indent = indent + ("│   " if branch == "├── " else "    ")

        # For true branch, use '├──'
        self.display_tree(node.true_branch, new_indent, "├── ")

        # For false branch, use '└──'
        self.display_tree(node.false_branch, new_indent, "└── ")

In [77]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Initialiser et entraîner l'arbre de décision symbolique
tree_xai = SymbolicDecisionTree()
tree_xai.fit(X_train, y_train)

# Évaluer la précision de l'arbre de décision
predictions = tree_xai.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.4f}%")

Accuracy: 100.0000%


In [4]:
def untrace_tree(example_class, trace_path, node, before):
    if node.result is not None:
        before.append((None, node.result))
        trace_path.append(before)
        print(f"End of tree at {node.result}")
    else:
        before_false = copy(before)
        before_false.append((node.feature, "!=", node.value))
        before_true = copy(before)
        before_true.append((node.feature, "==", node.value))
        untrace_tree(example_class, trace_path, node.false_branch, before_false)
        untrace_tree(example_class, trace_path, node.true_branch, before_true)


trace_path = []
untrace_tree(1, trace_path, tree_xai.tree, [])
trace_path

End of tree at -1
End of tree at 1
End of tree at -1


[[('Nationalite', '!=', 'Suisse'), ('Majeur?', '!=', 'oui'), (None, -1)],
 [('Nationalite', '!=', 'Suisse'), ('Majeur?', '==', 'oui'), (None, 1)],
 [('Nationalite', '==', 'Suisse'), (None, -1)]]

In [5]:
def filter_trace_path(class_to_not_keep, trace_path):
    l = []
    for path in trace_path:
        end_class = path[-1][1]
        if end_class != class_to_not_keep:
            l.append(path)
    return l


filtered_trace_path = filter_trace_path(1, trace_path)
filtered_trace_path

[[('Nationalite', '!=', 'Suisse'), ('Majeur?', '!=', 'oui'), (None, -1)],
 [('Nationalite', '==', 'Suisse'), (None, -1)]]

In [6]:
def count_rules(to_explain, trace_path):
    l = []
    for path in trace_path:
        n_r = 0
        for rule in path[:-1]:
            a, b, c = rule
            if b == "!=":
                if to_explain[a] != c:  # l'exemple valide la règle
                    pass
                else:  # l'exemple ne valise pas la règle
                    n_r += 1
            else:  # b == '=='
                if to_explain[a] == c:  # l'exemple valide la règle
                    pass
                else:  # l'exemple ne valise pas la règle
                    n_r += 1
        l.append(n_r)
    return np.array(l)


sample = X.iloc[0]
class_to_not_keep = y.iloc[0]
print(sample)
print(class_to_not_keep)
filtered_trace_path = filter_trace_path(1, trace_path)
n_r_list = count_rules(sample, filtered_trace_path)
np.argwhere(n_r_list == np.amax(n_r_list))

Adresse           Paris
Majeur?             oui
Nationalite    Francais
Name: 0, dtype: object
1


array([[0],
       [1]])

# Exercice 2


## Question 1: génération d'une base d'apprentissage


### Méthode LIME


In [7]:
# from copy import copy
# from sklearn.linear_model import Ridge


# # Nom des fonctions issus de la notation du cours
# class CustomExplainer:
#     def __init__(self, class_names, card_Z=10000):
#         self.class_names = class_names
#         self.card_Z = card_Z

#     def generate_z(self, x_ref):
#         Z = []  # list of all new exemple
#         weights = []
#         card_x_ref = len(x_ref)
#         sigma = int(
#             0.75 * np.sqrt(card_x_ref)
#         )  # on change maximum sigma nombre de mots dans la phrase
#         for _ in range(self.card_Z):
#             z_i = copy(x_ref)
#             nb_word_to_change = np.random.randint(0, sigma)
#             index_to_remove = np.random.randint(0, card_x_ref, size=nb_word_to_change)
#             for i in index_to_remove:
#                 splited_text_tmp[i] = ""
#             Z.append(" ".join(splited_text_tmp))
#             weights.append(1 - nb_word_to_change / card_x_ref)
#         return Z, weights

#     def explain_instance(self, text_instance, classifier_fn, num_features=None):
#         modified_text_instances, weights = self.generate_z(text_instance)
#         prob = classifier_fn(modified_text_instances).max(1)
#         clf = Ridge(fit_intercept=True)
#         clf.fit(
#             self.vectorizer.transform(modified_text_instances),
#             prob,
#             sample_weight=weights,
#         )
#         return pd.Series(clf.coef_, index=vectorizer.get_feature_names_out())


# explainer = CustomLimeTextExplainer(
#     class_names=newsgroups_test.target_names, vectorizer=vectorizer
# )
# coef = explainer.explain_instance(
#     newsgroups_test.data[idx], c.predict_proba, num_features=6
# )

## Methode LORE


In [38]:
from icecream import ic
from tqdm import tqdm
from distances import simple_match_distance


class CustomExplainer:
    def __init__(
        self,
        test_set: pd.DataFrame,
        model,
        distance,
        pc=0.5,
        pm=0.5,
        nb_genetics=100,
        N=1000,
    ):
        self.N = N
        self.pc = pc
        self.pm = pm
        self.nb_genetics = nb_genetics
        self.model = model
        self.distance = distance

        self.value_distribution = {}
        for col in test_set.columns:
            vc = test_set[col].value_counts(normalize=True)
            self.value_distribution[col] = (vc.index, vc.values)

    def geneticNeigh(self, x: pd.Series, same: bool):
        if same:
            fitness = self.fitness_same
        else:
            fitness = self.fitness_dif

        P = pd.DataFrame([x] * self.N)  # Population init
        M = self.evaluate(x, P, fitness)
        top_M = np.flip(M.argsort())
        for _ in tqdm(range(self.nb_genetics)):
            P = P.iloc[top_M].head(self.N)
            P = self.crossover(P)
            P = self.mutate(P)

            M = self.evaluate(x, P, fitness)
            top_M = np.flip(M.argsort())
        return P

    def evaluate(self, x, P: pd.DataFrame, fitness):
        x_hat = self.model.predict(x)
        M_list = []
        for _, z_i in P.iterrows():
            M_list.append(fitness(x, x_hat, z_i))
        return np.array(M_list)

    def fitness_same(self, x, x_hat, z_i):
        z_i_hat = self.model.predict(z_i)
        return (z_i_hat == x_hat) + (1 - self.distance(x, z_i)) - ((x == z_i).all())

    def fitness_dif(self, x, x_hat, z_i):
        z_i_hat = self.model.predict(z_i)
        return (z_i_hat != x_hat) + (1 - self.distance(x, z_i)) - ((x == z_i).all())

    def mutate(
        self,
        P: pd.DataFrame,
    ):
        idx_to_mutate = np.random.randint(0, len(P), size=int(len(P) * self.pm))
        mutations = P.iloc[idx_to_mutate].apply(self.mutate_one, axis=1)
        P_mutated = pd.concat((mutations, P))
        return P_mutated

    def mutate_one(self, parent):
        """
        Ici je fais deux mutation, il est possible d'en faire qu'une
        Le text du diapo en fait une, mais l'image en fait deux
        """
        attribute_list = parent.index
        att_1 = np.random.choice(attribute_list)
        att_2 = np.random.choice(attribute_list.drop(att_1))
        children = parent.copy()
        children[att_1] = np.random.choice(
            self.value_distribution[att_1][0], p=self.value_distribution[att_1][1]
        )
        children[att_2] = np.random.choice(
            self.value_distribution[att_2][0], p=self.value_distribution[att_2][1]
        )
        return children

    def crossover(self, P: pd.DataFrame):
        nb_parents = int(len(P) * self.pm)
        idx_parents = np.random.randint(0, len(P), size=(2, nb_parents // 2))
        crossovers = pd.concat(
            (
                self.crossover_one(P.iloc[idx_parents[0, i]], P.iloc[idx_parents[1, i]])
                for i in range(nb_parents // 2)
            )
        )
        P_mutated = pd.concat((crossovers, P))
        return P_mutated

    @staticmethod
    def crossover_one(parent_1, parent_2):
        attribute_list = parent_1.index
        att_1 = np.random.choice(attribute_list)
        att_2 = np.random.choice(
            attribute_list.drop(att_1)
        )  # on veut deux attribut différent
        att_list = [att_1, att_2]

        children_1 = parent_1.copy()
        children_1[att_list] = parent_2[att_list]

        children_2 = parent_2.copy()
        children_2[att_list] = parent_1[att_list]

        return pd.DataFrame((children_1, children_2))

custom_explainer = CustomExplainer(X, tree_xai, simple_match_distance)

In [35]:
Z_true = custom_explainer.geneticNeigh(X.iloc[0], True)

100%|██████████| 100/100 [04:09<00:00,  2.50s/it]


In [39]:
Z_false = custom_explainer.geneticNeigh(X.iloc[0], False)

100%|██████████| 100/100 [04:06<00:00,  2.47s/it]


In [63]:
pd.concat((Z, Z_hat), axis=1).iloc[:, -1].value_counts().idxmax()

1

Bon c'est super lent, aussi j'ai codé comme ça me venais un peu, au pire on utilisera l'[implémentation](https://github.com/riccotti/LORE/blob/master/neighbor_generator.py) mais c'est codé differement j'crois

Je sais pas c'est quoi les ordre de grandeur des hyprparamètre de l'algo génétique aussi

In [78]:
X_train

,Adresse,Majeur?,Nationalite
7,Montpellier,oui,Suisse
2,Montpellier,oui,Italien
4,Strasbourg,non,Italien
3,Paris,oui,Suisse
6,Strasbourg,oui,Francais


In [80]:
Z = pd.concat((Z_true, Z_false)).reset_index(drop=True)
Z_hat = pd.Series(tree_xai.predict(Z), name="Label")
# Z, _, Z_hat, _ = train_test_split(
#     X, y, test_size=0.01, random_state=42
# ) # gros bricolage ici, j'arrive pas à faire passer 
# Initialiser et entraîner l'arbre de décision symbolique
substitution_tree = SymbolicDecisionTree()
substitution_tree.fit(Z, Z_hat)

sample = X.iloc[0]
class_to_not_keep = y.iloc[0]
print(sample)
print(class_to_not_keep)
filtered_trace_path = filter_trace_path(1, trace_path)
n_r_list = count_rules(sample, filtered_trace_path)
np.argwhere(n_r_list == np.amax(n_r_list))

Adresse           Paris
Majeur?             oui
Nationalite    Francais
Name: 0, dtype: object
1


array([[0],
       [1]])